In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [4]:
digits = load_digits()

In [7]:
X=digits['data']

In [9]:
X=np.array([(X[i]-np.min(X[i]))/(np.max(X[i])-np.min(X[i])) for i in range(len(X))])

In [11]:
X.shape

(1797, 64)

In [13]:
X[0].shape

(64,)

In [15]:
y=digits['target']

In [19]:
y_one_hot=np.array([[0,0,0,0,0,0,0,0,0,0] for i in range(1) for j in range(y.shape[0])])

In [17]:
y_one_hot=np.array([[0,0,0,0,0,0,0,0,0,0] for i in range(1) for j in range(y.shape[0])])
for i in range(len(y)):    
        y_one_hot[i][y[i]]=1

In [117]:
X_train, X_test, y_label_train, y_label_test = train_test_split(X, y, test_size=0.2,random_state=42)
y_one_hot_label_train, y_one_hot_label_test= train_test_split(y_one_hot, test_size=0.2,random_state=42)

In [119]:
X_train.shape

(1437, 64)

In [121]:
X_test.shape

(360, 64)

In [123]:
y_one_hot_label_train.shape

(1437, 10)

In [125]:
y_one_hot_label_test.shape

(360, 10)

In [127]:
input_dim=64
output_dim=10
hidden_dim=100

In [129]:
W1 = np.random.uniform(-0.01, 0.01, size=(hidden_dim,input_dim))

In [131]:
W2 = np.random.uniform(-0.01, 0.01, size=(output_dim,hidden_dim))

In [133]:
def relu(x):
    return np.maximum(0 , x)

In [135]:
def sigmoid(x):
    x = np.array(x)
    x = np.clip(x, -500, 500)
    return 1/(1+np.exp(-x))

In [137]:
def softmax(x):
    """Compute the softmax of a vector x."""
    exp_x = np.exp(x - np.max(x))  # Subtract max(x) for numerical stability
    return exp_x / np.sum(exp_x)

In [139]:
def binary_crossentropy(y_true, y_pred):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    
    y_pred=y_pred.flatten()
    y_true=y_true.flatten()

    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = -np.mean(y_true* np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

In [141]:
def model_forward(x,W1,W2,output_activation="sigmoid"):
    x=np.reshape(x,(len(x),1))
    z1 = W1@x
    a1 = relu(z1)
    z2 = W2@a1
    if output_activation=="sigmoid":
        a2= sigmoid(z2)
    elif  output_activation=="softmax":
        a2= softmax(z2)
    else:
        pass
    return z1,a1,z2,a2

In [143]:
z1,a1,z2,a2=model_forward(X_train[0],W1,W2)

In [167]:
def model_backward(x,y_true,z1,a1,z2,a2,W1,W2,output_activation="sigmoid"):
    x=np.reshape(x,(len(x),1))
    y_true=np.reshape(y_true,(len(y_true),1))
    
    y_pred=a2
    
    epsilon = 1e-10  # Small value to avoid division by zero
    y_pred=np.clip(y_pred, epsilon, 1 - epsilon)
    
    
    dloss_dy_hat = ((y_pred - y_true )/(y_pred*(1-y_pred))).T
    
    if output_activation=="sigmoid":
        dy_hat_dz2 = np.eye(output_dim) * (y_pred * (1 - y_pred))
    elif output_activation=="softmax":
        dy_hat_dz2 = np.diagflat(y_pred) - y_pred@y_pred.T
        
    else:
        pass
        
    dz2_da1 = W2
    dloss_dz2= dloss_dy_hat @ dy_hat_dz2
    da1_dz1 = np.eye(hidden_dim)*(z1>0)*1
    dloss_dw1=(dloss_dz2@dz2_da1@da1_dz1).T@x.T
    #dloss_dw1=(dloss_dy_hat@dy_hat_dz2@dz2_da1@da1_dz1).T@x.T
    dloss_dw2 = dloss_dz2.T@a1.T
    
    return dloss_dw1, dloss_dw2

In [169]:
dloss_dw1, dloss_dw2 = model_backward(X_train[0],y_one_hot_label_train[0],z1,a1,z2,a2,W1,W2)

In [171]:
dloss_dw1.shape

(100, 64)

In [173]:
dloss_dw2.shape

(10, 100)

In [175]:
alpha=0.001
num_epochs=100
epoch=0
loss_history=[]

for epoch in range(num_epochs):
    epoch_loss=0
    for i in tqdm(range(len(X_train))):
        z1,a1,z2,a2=model_forward(X_train[i],W1,W2)
        loss=binary_crossentropy(y_one_hot_label_train[i],a2)
        epoch_loss=epoch_loss+loss
        gradW1,gradW2=model_backward(X_train[i],y_one_hot_label_train[i],z1,a1,z2,a2,W1,W2)
        W1=W1-alpha*gradW1
        W2=W2-alpha*gradW2
    epoch_loss=epoch_loss/len(X_train)
    loss_history.append(epoch_loss)
    print("Loss for epoch "+str(epoch+1)+" is "+str(epoch_loss))

100%|██████████| 1437/1437 [00:00<00:00, 7984.21it/s]


Loss for epoch 1 is 0.02445170098727777


100%|██████████| 1437/1437 [00:00<00:00, 11395.88it/s]


Loss for epoch 2 is 0.024219850264190056


100%|██████████| 1437/1437 [00:00<00:00, 10002.71it/s]


...

Loss for epoch 98 is 0.012069334918025467


100%|██████████| 1437/1437 [00:00<00:00, 11179.89it/s]


Loss for epoch 99 is 0.011995663970131404


100%|██████████| 1437/1437 [00:00<00:00, 10946.69it/s]

Loss for epoch 100 is 0.011926086086425208


In [176]:
predictions=[]
for i in range(len(X_test)):
    z1,a1,z2,a2=model_forward(X_test[i],W1,W2)
    y_pred=np.argmax(a2)
    y_test=np.argmax(y_one_hot_label_test[i])
    predictions.append(y_pred==y_test)

In [177]:
sum(np.array(predictions))/len(predictions)

0.975

### Neural Network Using Keras

In [179]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [180]:
input_layer = Input(X_train[0].shape)
hidden_layer = Dense(hidden_dim,activation='relu')(input_layer)
output_layer = Dense(output_dim,activation='sigmoid')(hidden_layer)

simple_one_hot_nn = Model(inputs=input_layer,outputs=output_layer)

In [181]:
simple_one_hot_nn.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │         6,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,510 (29.34 KB)

 Trainable params: 7,510 (29.34 KB)

 Non-trainable params: 0 (0.00 B)

In [182]:
simple_one_hot_nn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy')

In [183]:
history_oh_nn=simple_one_hot_nn.fit(X_train, y_one_hot_label_train, epochs=100, batch_size=1,shuffle=True)

Epoch 1/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 567us/step - loss: 0.4483
Epoch 2/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 498us/step - loss: 0.2703
Epoch 3/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 497us/step - loss: 0.2164
Epoch 4/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 497us/step - loss: 0.1718
Epoch 5/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 507us/step - loss: 0.1419
Epoch 6/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 489us/step - loss: 0.1200
Epoch 7/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step - loss: 0.1047
Epoch 8/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - loss: 0.0935
Epoch 9/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 541us/step - loss: 0.0802
Epoch 10/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 603us/step - loss: 0.0822
Epoch 11/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 566us/step - loss: 0.0688
Epoch 12/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 491us/step - loss: 0.0607
Epoch 13/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 491us/step - loss: 0.0588
Epoch 14/100
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 1s 494us/step - 

In [192]:
y_pred=simple_one_hot_nn.predict(X_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [194]:
y_pred.shape

(360, 10)

In [196]:
np.sum(np.argmax(y_pred,axis=1)==np.argmax(y_one_hot_label_test,axis=1))/len(y_pred)

0.9861111111111112